In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate
from langchain import OpenAI, ConversationChain

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import PyPDF2
import PyPDF2
import numpy as np 
import torch
device = torch.cuda.current_device() if torch.cuda.is_available() else None

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_id = "dslim/bert-base-NER"

# load the tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
# load the NER model from huggingface
model = AutoModelForTokenClassification.from_pretrained(
    model_id
)
# load the tokenizer and model into a NER pipeline
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="max",
    device=device
)

In [4]:
import os 
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

In [5]:
import pinecone 
index = pinecone.Index("mergers-and-acqs")
index_description = pinecone.describe_index("mergers-and-acqs")
index_stats_response = index.describe_index_stats()

TypeError: expected string or bytes-like object

In [6]:
index_stats_response

NameError: name 'index_stats_response' is not defined

In [33]:
# Extract text from the PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def get_all_pdf_files_in_directory(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

In [34]:
import pandas as pd
import numpy as np
import time
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('display.max_colwidth', 200)

In [35]:
import os
import PyPDF2
import pinecone
from sentence_transformers import SentenceTransformer

In [22]:
data_directory = "data/"
pdf_files = get_all_pdf_files_in_directory(data_directory)

In [36]:
pdf_files 

['data/Annual_Report_2021_22_Web_English_Versio.pdf',
 'data/BNR_Economic_Review_Vol_19.pdf',
 'data/MPFSS_March2023_Updated.pdf']

In [38]:
bnr_report_reader = UnstructuredPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
bnr_report_reader_data = bnr_report_reader.load()

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
conversation = ConversationChain(llm=llm, verbose=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\Leonce                                                                                  │
│ Nshuti\Desktop\portfolio\national-bank-of-Rwanda\embed\lib\site-packages\pdf2image\pdf2image.py: │
│ 568 in pdfinfo_from_path                                                                         │
│                                                                                                  │
│   565 │   │   env = os.environ.copy()                                                            │
│   566 │   │   if poppler_path is not None:                                                       │
│   567 │   │   │   env["LD_LIBRARY_PATH"] = poppler_path + ":" + env.get("LD_LIBRARY_PATH", "")   │
│ ❱ 568 │   │   proc = Popen(command, env=env, stdout=PIPE, stderr=PIPE)                           │
│   569 │   │                                                                                      │
│   570 │   │   try:                                                                               │
│   571 │   │   │   out, err = proc.communicate(timeout=timeout)                                   │
│                                                                                                  │
│ C:\Users\Leonce Nshuti\miniconda3\lib\subprocess.py:858 in __init__                              │
│                                                                                                  │
│    855 │   │   │   │   │   self.stderr = io.TextIOWrapper(self.stderr,                           │
│    856 │   │   │   │   │   │   │   encoding=encoding, errors=errors)                             │
│    857 │   │   │                                                                                 │
│ ❱  858 │   │   │   self._execute_child(args, executable, preexec_fn, close_fds,                  │
│    859 │   │   │   │   │   │   │   │   pass_fds, cwd, env,                                       │
│    860 │   │   │   │   │   │   │   │   startupinfo, creationflags, shell,                        │
│    861 │   │   │   │   │   │   │   │   p2cread, p2cwrite,                                        │
│                                                                                                  │
│ C:\Users\Leonce Nshuti\miniconda3\lib\subprocess.py:1311 in _execute_child                       │
│                                                                                                  │
│   1308 │   │   │                                                                                 │
│   1309 │   │   │   # Start the process                                                           │
│   1310 │   │   │   try:                                                                          │
│ ❱ 1311 │   │   │   │   hp, ht, pid, tid = _winapi.CreateProcess(executable, args,                │
│   1312 │   │   │   │   │   │   │   │   │   │    # no special security                            │
│   1313 │   │   │   │   │   │   │   │   │   │    None, None,                                      │
│   1314 │   │   │   │   │   │   │   │   │   │    int(not close_fds),                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_27160\3716189382.py:2 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Leonce                                          │
│ Nshuti\\AppData\\Local\\Temp\\ipykernel_27160\\3716189382.py'                                    │
│                 

In [ ]:
bnr_report_reader_data

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
pages = loader.load_and_split()

In [ ]:
# Pick a name for the new index
simple_index_name = 'stocks-trends'

In [ ]:
# Check whether the index with the same name already exists
if simple_index_name in pinecone.list_indexes():
    pinecone.delete_index(simple_index_name)

In [ ]:
# Create a new index
pinecone.create_index(name=simple_index_name, dimension=128)

In [ ]:
# Define nlp
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_named_entities(text_batch):
    # extract named entities using the NER pipeline
    extracted_batch = nlp(text_batch)
    entities = []
    # loop through the results and only select the entity names
    for text in extracted_batch:
        ne = [entity["word"] for entity in text]
        entities.append(ne)
    return entities

In [ ]:
pinecone.list_indexes()

In [ ]:
pinecone.describe_index("pinecone-index")

In [ ]:
pinecone.create_index("example-index", dimension=128, metric="euclidean", pods=4, pod_type="s1.x1")

In [ ]:
pinecone.create_index("example-index", dimension=128, source_collection="example-collection")

In [ ]:
pinecone.configure_index("my_index", pod_type="s1.x2")

In [ ]:
pinecone.describe_index("example-index")

In [ ]:
pinecone.configure_index("example-index", replicas=4)

In [ ]:
from pprint import pprint
# Define retriever
retriever = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Define index
index = pinecone.Index()

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
print (f'Now you have {len(texts)} documents')

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)

pinecone.create_index("national_bank_index", dimension=1024)

In [ ]:
pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")
index = pinecone.Index("example-index")

index_stats_response = index.describe_index_stats()

In [ ]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# create a new index 
index = client.create_index("bnr_index")

# add documents to the index
for text in texts:
    index.add_document(text)

# search for similar documents  
results = index.search("What are the top economic challenges facing Rwanda?", k=2)
for result in results:
    print(result)

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)
index = pinecone.Index("example-index")

In [ ]:
metadata_config = {'indexed': ['color']}

In [ ]:
# Use Pinecone to implement a vector store 
pinecone_index = Pinecone(index_name="example-index", metadata_config=metadata_config)

# Use OpenAI to implement an embedding model
openai_embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Use Weaviate to implement a vector store  

weaviate_index = Weaviate("http://localhost:8080", "bnr_index")

# Use FAISS to implement a vector store
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))

In [ ]:

# Use approximate nearest neighbor search to find similar documents
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)